In [1]:
%load_ext ipython_sparql_pandas
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper('http://DESKTOP-CELL0BF:7200/repositories/20qsmall')
sparql.setReturnFormat(JSON)

In [2]:
import rdflib
import html
from SPARQLWrapper import SPARQLWrapper, JSON
g = rdflib.Graph().parse('./fb15k-wikitop2021-yago-facts.nt', format='ntriples')
g += rdflib.Graph().parse('./fb15k-wikitop2021-yago-full-types.nt', format='ntriples')

from rdflib.namespace import RDF
for line in open('fb15k-wikitop2021.tsv').readlines()[1:]:
    subject = rdflib.URIRef( line.split('\t')[6][1:-1] ) # take the URI in column 6
    g.add( (subject, RDF.type, rdflib.URIRef('http://example.com/popularEntity')) )
    
g.remove((None, rdflib.URIRef('http://schema.org/image') , None))
g.remove((None, rdflib.URIRef('http://schema.org/url') , None))


<Graph identifier=N6a531f84605b45a4bfea8a0331c0df70 (<class 'rdflib.graph.Graph'>)>

In [4]:
#from the query results extract a list that contains the counts after splitting on certain attributes
import re
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .', '?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def prepareProps(popEntities):
    Distribution = {}
    for i in popEntities:
        Distribution[i]= 1/len(popEntities)
    return Distribution


def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    #print(listOfObj[0])
    return(listOfObj,listOfPred)


  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            group by ?p ?o 
            ORDER BY ABS( {left/2} - ?count )
            """) 
        
    return FilterQuestion

In [5]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
            {PosFilters}{NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))
    
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
            {PosFilters}
            {NegFilters} 
            }}
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    results = []
    for r in qres["results"]["bindings"]:
        results.append(r["s"]["value"])
    return results


In [6]:
# resets are variables
def game_reset():
    left = numberleft("","")
    FilterQuestion = updateQuery(left, "","")
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [7]:
def prepareProps(popEntities):
    Distribution = {}
    for i in popEntities:
        Distribution[i] = 1
        #Distribution[i]= 1/len(popEntities)
    return Distribution
    

def findPosAndNeg(listOfObj, listOfPred, index):
    Filter = '?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.'
    negFilter='FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}'
    EntitiesPositive = popentities(Filter, "")
    EntitiesNegative = popentities("", negFilter)
    return (EntitiesPositive, EntitiesNegative)
    
                       
def findSplits():
    possibleObj=[]
    possiblePred =[]
    count = []
    query =  f"""
            select (count(?s) as ?count)   ?o ?p
            {{               
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
        ?s ?p ?o.
            }}
        group by  ?o ?p
        ORDER BY desc ( ?count )
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        possibleObj.append(r["o"]["value"])
        possiblePred.append(r["p"]["value"])
        count.append(r["count"]["value"])
   
    return (possibleObj, possiblePred, count)

def determineBestSplit(possibleObj, possiblePred, count):
    bestSplit=-1
    indexBestSplit = -1
    #branches = range(2,250)
    #for i in random.sample(branches, 40): 
    for i in range(350):
        #Fix The nummerical thing here::: (skip over i if nummer)
        if str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>') in AttrHistory:
            continue
        else:
            Yes = pos.get(str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>'))
            No = negs.get(str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>'))
            OccurenceYes = len(Yes)
            OccurenceNo=len(No)
            """Yes, No = findPosAndNeg(possibleObj, possiblePred, i)
            OccurenceYes=len(Yes)
            OccurenceNo=len(No)"""
            WeightYes=0
            WeightNo=0
            for y in Yes:
                WeightYes+=Distribution[y]
            for n in No:
                WeightNo+=Distribution[n]
            SumWeights=WeightYes+WeightNo
            ProbabilityYes=WeightYes/SumWeights
            ProbabilityNo=WeightNo/SumWeights
            Entropy = - (ProbabilityYes * math.log(ProbabilityYes+0.0001,2) + ProbabilityNo * math.log(ProbabilityNo+0.0001,2))
            #print(str(possiblePred[i]) +'> <'+str(possibleObj[i]), Entropy)
            if Entropy > bestSplit:
                bestSplit=Entropy
                indexBestSplit=i
    #print(bestSplit, possibleObj[indexBestSplit], possiblePred[indexBestSplit])
    return(indexBestSplit, bestSplit)

def WrongAnswerRate(answer, chance):
    if randint(0,100)<chance:
        print("haha fooled!")
        if answer:
            answer=False
        else:
            answer=True
    return(answer)
        
def wrongAnswerList( amount):
    l = []
    i = -1
    for i in range(20-amount):
        l.append(True)
    for i in range(amount):
        l.append(False)
    random.shuffle(l)
    return(l, i)

def retrieveAnswer(answer, l, i):
    i+=1
    if l[i]:
        return(answer, i)
    else:
        print("wrong answer incoming")
        return(not answer, i)
"""preparePops
enter loop
find splits, determine best split, generate question (update probabilities)"""
                             
                             
        


'preparePops\nenter loop\nfind splits, determine best split, generate question (update probabilities)'

In [8]:
def generateHumanQuestion(listOfObj, listOfPred, index, subject, Distribution,itteration):  
    AttrHistory.append('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>')
    answer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]}?' )
    Yes, No = findPosAndNeg(listOfObj, listOfPred, index)
    if answer.startswith('y'):   
        for y in Yes:
            Distribution[y]= Distribution[y]*0.9
        for n in No:
            Distribution[n]= Distribution[n]*0.1
            
    else:
        for y in Yes:
            Distribution[y] = Distribution[y]*0.1
        for n in No:
            Distribution[n] = Distribution[n]*0.9
    return Distribution

"""def allSubjects(qres, ind):
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")"""

def allSubjects(qres, ind):
    attributes=[]
    subject=qres[ind]
    query = f"""
    select  ?p ?o
    where {{
    <{subject}> ?p ?o .}}"""
    results = g.query(query)
    for b in results:
        attributes.append(f'{subject}, {b.p}, {b.o}')
    return(subject, attributes)

def loadSubjects():

    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
    }}
    """
    results = g.query(query)
    ents=[]
    for b in results:
        ents.append(f'{b.s}')
    return ents
    
def pickSubject(qres):
    ind=randint(0,134)
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

def standartize(Distribution):
    standard=sum(Distribution.values())
    for d in Distribution.keys():
        Distribution[d] = Distribution.get(d) / standard
    return Distribution

In [50]:
def generateBotQuestionManipulate(listOfObj, listOfPred, index, subject, Distribution,itteration, attribu, pos, negs):
    AttrHistory.append('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>')
    #print(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]}?' )
    if f'{subject}, {listOfPred[index]}, {listOfObj[index]}'in attribu:
        answer=True
    else:
        answer=False
    """if itteration in [20]:
        answer = not answer"""
    #answer = WrongAnswerRate(answer, 10)
    answer, initilaizeCount = retrieveAnswer(answer, answerPattern,i-1)
    #print(answer)
    
    #Yes, No = findPosAndNeg(listOfObj, listOfPred, index)
    #Yes = pos.values()
    #No = negs.values
    Yes = pos.get(str('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>'))
    No = negs.get(str('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>'))
    """OccurenceYes=len(Yes)
    OccurenceNo=len(No)
    NEntities=OccurenceYes+OccurenceNo
    ProbabilityYes=OccurenceYes/NEntities
    ProbabilityNo=OccurenceNo/NEntities"""

    if answer:   
        for y in Yes:
            Distribution[y]= Distribution[y]*0.9
        for n in No:
            Distribution[n]= Distribution[n]*0.1
            
    else:
        for y in Yes:
            Distribution[y] = Distribution[y]*0.1
        for n in No:
            Distribution[n] = Distribution[n]*0.9
            
    #print(Distribution[subject[1:-1]])
    Distribution=standartize(Distribution)
    return Distribution

In [52]:
def loadAllAttributes(possibleObj,possiblePred):
    EntsPerAttribute={}
    EntsWithoutAttribute={}
    for i in range(350):
        Yes, No = findPosAndNeg(possibleObj, possiblePred, i)

        attr=str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>')
        EntsPerAttribute[attr]=Yes
        EntsWithoutAttribute[attr]=No
    return(EntsPerAttribute, EntsWithoutAttribute)


"""possibleObj, possiblePred, count = findSplits()
pos, negs= loadAllAttributes(possibleObj,possiblePred)"""

'possibleObj, possiblePred, count = findSplits()\npos, negs= loadAllAttributes(possibleObj,possiblePred)'

In [53]:
%%time
#without printing results

from random import randint
import random
import math
import time 
wincount = 0
gamesplayed = 0
wrongAnswersPerQuestion = 3
totalgames = 134
Entrop_over_game = []
list_losses=[]
list_wins=[]

possibleObj, possiblePred, count = findSplits()
pos,negs = loadAllAttributes(possibleObj,possiblePred)
qres = loadSubjects()

while gamesplayed < totalgames:
    answerPattern, initilaizeCount = wrongAnswerList(wrongAnswersPerQuestion)
    Entropies=[]
    AttrHistory= ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    # Resets all variables
    i=0
    outofguess = []
    # pick subject
    #subject = pickSubject(qres)
    subject, attribu = allSubjects(qres, gamesplayed)
    print("\n" , subject)
    popularEntities =  popentities("\n".join(PosAttr),"")
    Distribution = prepareProps(popularEntities)
    
    while i < 20:

        # list of Pred and Obj for questions
        #listOfObj ,listOfPred = extractCountAndAttr(qres)

        # if only 1 popular entity is left break


        # ask question and add filters to query
        print('Question: %s' %(i+1))

        indexSplit, Entropy=determineBestSplit(possibleObj, possiblePred, count)
        Distribution = generateBotQuestionManipulate(possibleObj, possiblePred, indexSplit,subject, Distribution, i, attribu, pos, negs)
        i+=1
        Entropies.append(Entropy)
    # Prints answer
    #print(Distribution)
    highProb=-1
    for j in Distribution.values():
        if j > highProb:
            highProb=j
    answer = list(Distribution.keys())[list(Distribution.values()).index(highProb)]
    print("The Answer is: ", answer, "With a Probability of: ", highProb)
    print(subject)
    if str(answer.split("/")[-1]) == str(subject.split("/")[-1]):
        wincount +=1
        #list_wins.append(answerPattern)
    #else:
        
        #list_losses.append(answerPattern)
    gamesplayed+=1
    Entrop_over_game.append(Entropies)
    #time.sleep(0.5)
    
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")


 http://yago-knowledge.org/resource/Elizabeth_II
Question: 1
Question: 2
Question: 3
Question: 4
Question: 5
Question: 6
Question: 7
Question: 8
Question: 9
Question: 10
wrong answer incoming
Question: 11
Question: 12
Question: 13
Question: 14
wrong answer incoming
Question: 15
Question: 16
wrong answer incoming
Question: 17
Question: 18
Question: 19
Question: 20
The Answer is:  http://yago-knowledge.org/resource/Elizabeth_II With a Probability of:  0.9999972489178288
http://yago-knowledge.org/resource/Elizabeth_II

 http://yago-knowledge.org/resource/Joe_Biden
Question: 1
Question: 2
Question: 3
Question: 4
Question: 5
wrong answer incoming
Question: 6
Question: 7
wrong answer incoming
Question: 8
Question: 9
Question: 10
Question: 11
Question: 12
Question: 13
Question: 14
Question: 15
Question: 16
wrong answer incoming
Question: 17
Question: 18
Question: 19
Question: 20
The Answer is:  http://yago-knowledge.org/resource/Joe_Biden With a Probability of:  0.9994570078964615
http://yag

In [ ]:
#For HUMOONS
from random import randint
import random
import math
import time 
gamesplayed = 0
totalgames = 1
Entrop_over_game = []
list_losses=[]
list_wins=[]
subject=""
possibleObj, possiblePred, count = findSplits()
pos,negs = loadAllAttributes(possibleObj,possiblePred)

while gamesplayed < totalgames:
    Entropies=[]
    AttrHistory= ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    # Resets all variables
    i=0
    outofguess = []
    # pick subject
    #subject = pickSubject(qres)
    popularEntities =  popentities("\n".join(PosAttr),"")
    Distribution = prepareProps(popularEntities)
    
    while i < 20:

        # list of Pred and Obj for questions
        #listOfObj ,listOfPred = extractCountAndAttr(qres)

        # if only 1 popular entity is left break


        # ask question and add filters to query
        print('Question: %s' %(i+1))

        indexSplit, Entropy=determineBestSplit(possibleObj, possiblePred, count)
        #Distribution = generateBotQuestionManipulate(possibleObj, possiblePred, indexSplit,subject, Distribution, i)
        Distribution = generateHumanQuestion(possibleObj, possiblePred, indexSplit,subject, Distribution, i)
        i+=1
        Entropies.append(Entropy)
    # Prints answer
    #print(Distribution)
    highProb=-1
    for j in Distribution.values():
        if j > highProb:
            highProb=j
    answer = list(Distribution.keys())[list(Distribution.values()).index(highProb)]
    print("The Answer is: ", answer, "With a Probability of: ", highProb)
    print(subject)
    if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
        wincount +=1
        #list_wins.append(answerPattern)
    #else:
        
        #list_losses.append(answerPattern)
    gamesplayed+=1
    Entrop_over_game.append(Entropies)
    time.sleep(0.5)
    
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

In [15]:
print("Output1:", sorted(Distribution.values())[-2])
print(Distribution['http://yago-knowledge.org/resource/William_Shakespeare'])

Output1: 0.0015009463529699922
0.00016677181699666584


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xax = numpy.array(range(0,134))
for i in dis_over_time:
    yax=[]
    for j in i:
        yax.append(j)
    plt.plot(xax, yax)
plt.show

color = plt.cm.rainbow(np.linspace(0, 1, len(dis_over_time)))
for i, c in zip(range(len(dis_over_time)), color):
    yax=dis_over_time[i]
    plt.plot(xax, yax, c=c)

In [ ]:
yax = []
xax = numpy.array(range(0,134))

for j in range(9):
    yax = []
    for i in analyse_dis[j].values():
        yax.append(i)
    plt.plot(xax, yax)
    plt.show()

In [ ]:
#without printing results
#interesting experiment:
from random import randint
import random
import math
import time 
itterations = 18
patternsIt = 0
performances = []
lists = []
wincount = 0
gamesplayed = 0
#wrongAnswersPerQuestion = 4
totalgames = 134
dictperform={}

"""Entrop_over_game = []
list_losses=[]
list_wins=[]"""

possibleObj, possiblePred, count = findSplits()
pos,negs = loadAllAttributes(possibleObj,possiblePred)
qres = loadSubjects()

while patternsIt<itterations:
    answerPattern=[True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
    answerPattern[patternsIt] = False
    answerPattern[patternsIt+1] = False
    answerPattern[patternsIt+2] = False
    print(answerPattern)
    initilaizeCount=0
    wincount = 0
    gamesplayed = 0    
    #answerPattern, initilaizeCount = wrongAnswerList(wrongAnswersPerQuestion)
    popularEntities =  popentities("\n".join(PosAttr),"")
    patternsIt+=1
    
    while gamesplayed < totalgames:
        
        Entropies=[]
        AttrHistory= ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
        # Resets all variables
        i=0
        outofguess = []
        # pick subject
        #subject = pickSubject(qres)
        subject, attribu = allSubjects(qres, gamesplayed)
        print("\n" , subject)
        Distribution = prepareProps(popularEntities)

        while i < 20:
            indexSplit, Entropy=determineBestSplit(possibleObj, possiblePred, count)
            Distribution = generateBotQuestionManipulate(possibleObj, possiblePred, indexSplit,subject, Distribution, i, attribu, pos, negs)
            i+=1
            Entropies.append(Entropy)
        highProb=-1
        for j in Distribution.values():
            if j > highProb:
                highProb=j
        answer = list(Distribution.keys())[list(Distribution.values()).index(highProb)]
        print("The Answer is: ", answer, "With a Probability of: ", highProb)
        print(subject)
        if str(answer.split("/")[-1]) == str(subject.split("/")[-1]):
            wincount +=1
            #list_wins.append(answerPattern)
        #else:

            #list_losses.append(answerPattern)
        gamesplayed+=1
        #time.sleep(0.5)
    lists.append(answerPattern)
    performances.append(wincount)
    dictperform[wincount] =answerPattern
    print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

In [8]:
Distribution={"oma":2, "opa":3, "mama":1}
#
Distribution = standartize(Distribution)
print(Distribution)

{'oma': 0.3333333333333333, 'opa': 0.5, 'mama': 0.16666666666666666}


In [66]:
import numpy
import matplotlib.pyplot as plt
sample = numpy.random.normal(loc=5, scale=5, size=20)
points = numpy.round(sample)
print(points)

[-1. 11.  1.  8. -4. 10.  5.  6.  4. 13.  9.  6. -0.  8.  7. -1.  5. 10.
  7. 13.]


In [11]:
#print(lists)

print(max(performances))
print(min(performances))
#dictperform[wincount] =answerPattern

import numpy
import matplotlib.pyplot as plt
print(len(performances))

134
133
2


In [ ]:
print(performances)

In [21]:
for i in range(len(lists)):
    print(performances[i], lists[i], "\n")

131 [False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 

123 [True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 

130 [True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 

129 [True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 

134 [True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 

134 [True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 

133 [True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True] 

130 [True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, Tr

In [4]:
FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{

             ?s ?p ?o .
            }} 
            group by ?p ?o 
            ORDER BY DESC
            """) 

sparql.setQuery(FilterQuestion)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert() 
res = qres["results"]["bindings"]
print(res)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>